## Geologische Geschichte des Geländes

1. Ablagerung der ersten Sequenz:
    *  A
    *  B
    *  C 
    *  D
  
2. Störung F1

3. Ablagerung der 2. Sequenz
    * G
    * H
4. Störung F2

In [1]:
# These three lines are necessary only if GemPy is not installed
import sys, os
sys.path.append('../..')
sys.path.append('../../../gempy/')

# Importing GemPy
import gempy as gp

# Importing aux libraries
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pn
import matplotlib
import theano
import qgrid
import pickle

# Imort SandBox
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN"
#import sandbox.sandbox as sb


### Initializing the model:

The first step to create a GemPy model is create a gempy.Model object that will contain all the other data structures and necessary functionality.

In addition for this example we will define a regular grid since the beginning. This is the grid where we will interpolate the 3D geological model. GemPy comes with an array of different grids for different pourposes as we will see below. For visualization usually a regular grid is the one that makes more sense.

In [2]:
geo_model = gp.create_model('Geological_Model1')
geo_model = gp.init_data(geo_model, extent= [0, 4000, 0, 2775, 200, 1200], resolution=[100, 10, 100])

Active grids: ['regular']


GemPy core code is written in Python. However for efficiency (and other reasons) most of heavy computations happend in optimize compile code, either C or CUDA for GPU. To do so, GemPy rely on the library theano. To guarantee maximum optimization theano requires to compile the code for every Python kernel. The compilation is done by calling the following line at any point (before computing the model):

In [3]:
gp.set_interpolator(geo_model, theano_optimizer='fast_run', verbose=[])

[]
[], Categories (3, object): [Erosion, Onlap, Fault]
is_erosion []
[]
[], Categories (3, object): [Erosion, Onlap, Fault]
is_erosion []
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!


### Creating figure:

GemPy uses matplotlib and pyvista-vtk libraries for 2d and 3d visualization of the model respectively. One of the design decisions of GemPy is to allow real time construction of the model. What this means is that you can start adding input data and see in real time how the 3D surfaces evolve. Lets initialize the visualization windows.

The first one is the 2d figure. Just place the window where you can see it (maybe move the jupyter notebook to half screen and use the other half for the renderers).

In [4]:
#Visualization Widgets - Conflicts with bokeh visualization
from gempy.plot import visualization_2d_pro as vv
#from gempy.plot import vista

In [5]:
%matplotlib qt5

map_view = vv.Plot2D(geo_model)
map_view.create_figure((15, 8))

profile_view = vv.Plot2D(geo_model)
profile_view.create_figure((15, 8))

(<Figure size 3000x1600 with 0 Axes>, array([], shape=(0, 0), dtype=object))

#### Add model section

In the 2d renderer we can add several cross section of the model. In this case, for simplicity we are just adding one perpendicular to z.

In [6]:
# In this case perpendicular to the z axes
ax = map_view.add_section(direction='z')

ax2 = profile_view.add_section(direction='y')


#### Loading geological map image:

Remember that gempy is simply using matplotlib and therofe the ax object created above is a standard matplotlib axes. This allow to manipulate it freely. Lets load an image with the information of geological map

In [7]:
# Reading image
img = mpimg.imread('geological_model.png')
# Plotting it inplace
ax.imshow(img, origin='upper', alpha=.8, extent = (0, 4000, 0,2775))



In [8]:

ax2.set_xlim(geo_model.grid.regular_grid.extent[0], geo_model.grid.regular_grid.extent[1])
ax2.set_ylim(geo_model.grid.regular_grid.extent[4], geo_model.grid.regular_grid.extent[5])

(200.0, 1200.0)

## Building the model

Now that we have everything initialize we can start the construction of the geological model. 
### Cycle1:

#### Surfaces

GemPy is a surface based interpolator. This means that all the input data we add has to be refered to a surface. The surfaces always mark the bottom of a unit. By default GemPy surfaces are empty:

In [9]:
geo_model.series

,order_series,BottomRelation
Default series,1,Erosion


In [10]:
geo_model.add_series('Cycle1')

,order_series,BottomRelation
Default series,1,Erosion
Cycle1,2,Erosion


In [11]:
geo_model.delete_series('Default series')

,order_series,BottomRelation
Cycle1,1,Erosion


In [12]:
#geo_model.set_is_fault(['Fault1', 'Fault2'])

We can create the first surfaces for the Cycle1 of the sedimentary layers:

In [13]:
geo_model.add_surfaces(['D','C','B', 'A'])

,surface,series,order_surfaces,color,id
0,D,Cycle1,1,#015482,1
1,C,Cycle1,2,#9f0052,2
2,B,Cycle1,3,#ffbe00,3
3,A,Cycle1,4,#728f02,4


Series is the object that contains the properties associated with each independent scalar field. The name by default is "Default series" but we can rename it and create new ones as we advance in the constructin of the model

In [14]:
geo_model.series

,order_series,BottomRelation
Cycle1,1,Erosion


In [15]:
#geo_model.rename_series(['Cycle1'])

Now we can start adding data. GemPy input data consist on surface points and orientations (perpendicular to the layers). The 2D plot gives you the X and Y coordinates when hovering the mouse over. We can add a surface point as follows:

In [16]:
#surface B
geo_model.add_surface_points(X=429, Y=2482, Z=400, surface='B')
geo_model.add_surface_points(X=483, Y=2180, Z=400, surface='B')
geo_model.add_surface_points(X=523, Y=682, Z=500, surface='B')
geo_model.add_surface_points(X=300, Y=12, Z=600, surface='B')

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
0,429.0,2482.0,400.0,0.503643,0.750100,0.479857,B,Cycle1,3,1,0.000001
1,483.0,2180.0,400.0,0.514574,0.688966,0.479857,B,Cycle1,3,1,0.000001
2,523.0,682.0,500.0,0.522671,0.385728,0.500100,B,Cycle1,3,1,0.000001
3,300.0,12.0,600.0,0.477529,0.250100,0.520343,B,Cycle1,3,1,0.000001


In [17]:
#surface C
geo_model.add_surface_points(X=780, Y=2493, Z=500, surface='C')
geo_model.add_surface_points(X=832, Y=2133, Z=500, surface='C')
geo_model.add_surface_points(X=850, Y=665, Z=600, surface='C')
geo_model.add_surface_points(X=952, Y=192, Z=600, surface='C')
#Surface D
geo_model.add_surface_points(X=961, Y=1635, Z=800, surface='D')
geo_model.add_surface_points(X=1107, Y=991, Z=800, surface='D')


,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
8,961.0,1635.0,800.0,0.551994,0.577186,0.540406,D,Cycle1,1,1,0.000001
9,1107.0,991.0,800.0,0.581418,0.447399,0.540406,D,Cycle1,1,1,0.000001
4,780.0,2493.0,500.0,0.515517,0.750100,0.479947,C,Cycle1,2,1,0.000001
5,832.0,2133.0,500.0,0.525997,0.677549,0.479947,C,Cycle1,2,1,0.000001
6,850.0,665.0,600.0,0.529624,0.381700,0.500100,C,Cycle1,2,1,0.000001
7,952.0,192.0,600.0,0.550181,0.286376,0.500100,C,Cycle1,2,1,0.000001
0,429.0,2482.0,400.0,0.444780,0.747883,0.459794,B,Cycle1,3,1,0.000001
1,483.0,2180.0,400.0,0.455662,0.687021,0.459794,B,Cycle1,3,1,0.000001
2,523.0,682.0,500.0,0.463724,0.385126,0.479947,B,Cycle1,3,1,0.000001
3,300.0,12.0,600.0,0.418782,0.250100,0.500100,B,Cycle1,3,1,0.000001


The minimum amount of data to interpolate anything in gempy is

a) 2 surface points per surface
b) One orientation per series.

Lets add an orientation for the first cycle:

In [18]:
# Adding orientation
geo_model.add_orientations(X=429, Y=2482, Z=400, surface='B', orientation = [74.0, 31.89, 1.0])


,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,429.0,2482.0,400.0,0.44478,0.747883,0.459794,0.507825,0.145617,0.849064,31.89,74.0,1.0,B,Cycle1,3,1,0.01


In [19]:
# Plot in 2D
map_view.plot_data(ax, direction='z')

Now we have enough data for finally interpolate!

In [20]:
gp.compute_model(geo_model)
#export and send model to sandbox
geo_model.save_model_pickle(path='temp.pickle')

../../../gempy\gempy\core\solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')


True

In [21]:
# In 2D
#profile_view.remove(ax2)
profile_view.plot_contacts(ax2, direction='y', cell_number=5)
profile_view.plot_lith(ax2, direction='y', cell_number=5)
# In 3D
#p3d.plot_surfaces()

0 3


In [ ]:
gp.plot.plot_3D(geo_model)

### Fault1 :
So far the model is simply a depositional unit. GemPy allows for unconformities and faults to build complex models. This input is given by categorical data. In general:

input data (surface points/ orientations) <belong to< surface <belong to< series

And series can be a fault---i.e. offset the rest of surface--- or not. We are going to show how to add a fault.

First we need to add a series:

In [22]:
geo_model.add_series(['Fault1'])

,order_series,BottomRelation
Cycle1,1,Erosion
Fault1,2,Erosion


In [23]:
geo_model.modify_order_series(1, 'Fault1')

,order_series,BottomRelation
Fault1,1,Erosion
Cycle1,2,Erosion


In [24]:
#geo_model.reorder_series(['Fault1', 'Cycle1'])

Then define that is a fault:

But we also need to add a new surface:

In [25]:
geo_model.add_surfaces(['F1'])

,surface,series,order_surfaces,color,id
0,D,Cycle1,1,#015482,1
1,C,Cycle1,2,#9f0052,2
2,B,Cycle1,3,#ffbe00,3
3,A,Cycle1,4,#728f02,4
4,F1,Cycle1,5,#443988,5


And finally assign the new surface to the new series/fault

In [26]:
gp.map_series_to_surfaces(geo_model, {'Fault1':'F1'})

,surface,series,order_surfaces,color,id
4,F1,Fault1,1,#443988,1
0,D,Cycle1,1,#015482,2
1,C,Cycle1,2,#9f0052,3
2,B,Cycle1,3,#ffbe00,4
3,A,Cycle1,4,#728f02,5


In [27]:
geo_model.set_is_fault('Fault1')

Fault colors changed. If you do not like this behavior, set change_color to False.


,isFault,isFinite
Fault1,True,False
Cycle1,False,False


Now we can just add input data as before (remember the minimum amount of input data to compute a model):

In [29]:
# Add input data of the fault
geo_model.add_surface_points(X=1213, Y=309, Z=500, surface='F1')
geo_model.add_surface_points(X=1250, Y=1864, Z=700, surface='F1')

#Add orientation
geo_model.add_orientations(X=1250, Y=1864, Z=700, surface='F1', orientation = [92.0, 90.0, 1.0])

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
1,1250.0,1864.0,700.0,0.595828,0.623337,0.520253,0.999391,-0.034899,1.000061e-12,90.00,92.0,1.0,F1,Fault1,1,1,0.01
0,429.0,2482.0,400.0,0.430370,0.747883,0.459794,0.507825,0.145617,8.490639e-01,31.89,74.0,1.0,B,Cycle1,4,2,0.01


In [31]:
# Compute
gp.compute_model(geo_model)

# Plot
map_view.plot_data(ax, direction='z')

profile_view.remove(ax2)
profile_view.plot_lith(ax2, cell_number=5)
#p3d.plot_structured_grid(opacity=.2, annotations = {2: 'surface1', 3:'surface2', 4:'surface3', 5:'basement'})

As you can see now instead of having dipping layers we have a sharp jump. But there is no information on the other side of the fault. That is because we now are going to add the information on the afected block. 

In [32]:
#surface B
geo_model.add_surface_points(X=1450, Y=2550, Z=500, surface='B')
geo_model.add_surface_points(X=1640, Y=270, Z=600, surface='B')

#surface C
geo_model.add_surface_points(X=1890, Y=2076, Z=600, surface='C')
geo_model.add_surface_points(X=2010, Y=155, Z=700, surface='C')
#Surface D
geo_model.add_surface_points(X=2810, Y=2560, Z=600, surface='D')
geo_model.add_surface_points(X=2890, Y=860, Z=700, surface='D')


,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
10,1213.0,309.0,500.0,0.426355,0.311490,0.480795,F1,Fault1,1,1,0.000001
11,1250.0,1864.0,700.0,0.433498,0.611683,0.519405,F1,Fault1,1,1,0.000001
8,961.0,1635.0,800.0,0.377706,0.567475,0.538710,D,Cycle1,2,2,0.000001
9,1107.0,991.0,800.0,0.405892,0.443150,0.538710,D,Cycle1,2,2,0.000001
16,2810.0,2560.0,600.0,0.734656,0.746046,0.500100,D,Cycle1,2,2,0.000001
17,2890.0,860.0,700.0,0.750100,0.417861,0.519405,D,Cycle1,2,2,0.000001
4,780.0,2493.0,500.0,0.342764,0.733112,0.480795,C,Cycle1,3,2,0.000001
5,832.0,2133.0,500.0,0.352803,0.663614,0.480795,C,Cycle1,3,2,0.000001
6,850.0,665.0,600.0,0.356278,0.380216,0.500100,C,Cycle1,3,2,0.000001
7,952.0,192.0,600.0,0.375969,0.288903,0.500100,C,Cycle1,3,2,0.000001


In [33]:
# Compute
gp.compute_model(geo_model)
#export and send model to sandbox
geo_model.save_model_pickle(path='temp.pickle')

# Plot
map_view.plot_data(ax, direction='z', cell_number=11)

profile_view.remove(ax2)
profile_view.plot_lith(ax2, cell_number=5)


Now all the first sequence is complete, with the deposition of some sedimentary layes and its posterior faulting

## Iteractive DataFrame

### Activating Qgrid

Qgrid is only a gempy dependency. Therefore to use it, first we need to activate it in a given model by using:

In [ ]:
gp.activate_interactive_df(geo_model)

This will create the interactive dataframes objects. This dataframes are tightly linked to the main dataframes of each data class.

#### Series

In [ ]:
geo_model.qi.qgrid_se

#### Faults

In [ ]:
geo_model.qi.qgrid_fa

#### surfaces

In [ ]:
geo_model.qi.qgrid_fo

#### surface points

In [ ]:
geo_model.qi.qgrid_in

#### Orientations

In [ ]:
geo_model.qi.qgrid_or

Remember we are always changing the main df as well!

### Plot

In [34]:
# Compute
gp.compute_model(geo_model)

# Plot
gp.plot.plot_3D(geo_model)


holding... Use vtk.resume to go back to the interactive window


### Sandbox 

In [ ]:
#export and send model to sandbox
geo_model.save_model_pickle(path='temp.pickle')